In [1]:
import os
import json
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# final result (93.27%)
result_file = "../../results/tabmwp/chameleon_chatgpt_test_cache.jsonl"
data_file = "../../data/tabmwp/problems_test.json"

In [3]:
inventory = ['program_generator',
 'program_verifier',
 'program_executor',
 'answer_generator',
 'row_lookup',
 'column_lookup',
 'table_verbalizer',
 'knowledge_retrieval',
 'solution_generator',
] 

In [ ]:
prob_data = json.load(open(data_file))

with open(result_file) as f:
    results = {}
    for line in f:
        try:
            data = json.loads(line)
            results[data["pid"]] = data
        except:
            pass
        
test_pids = list(results.keys())
len(test_pids)

# construct pandas data
prob_data = pd.DataFrame(prob_data).T
res_pd = prob_data[prob_data.index.isin(test_pids)]  # test set

# update data
for index, row in res_pd.iterrows():
    result = results[index]
    
    res_pd.loc[index, 'true_false'] = result['true_false']
    
    modules = result["modules:output"]
    
    #print("before", modules)
    if "program_generator_and_verifier" in modules:
        nested = []
        for module in modules:
            if module == "program_generator_and_verifier":
                nested.append(["program_generator", "program_verifier"])
            else:
                nested.append([module])
        modules = [m for sublist in nested for m in sublist]
    #print("after", modules)
    #print("")
    
    res_pd.loc[index, 'modules'] = str(modules)
    res_pd.loc[index, 'num_modules'] = int(len(modules))
    
    for module in inventory:
        res_pd.loc[index, module] = True if module in modules else False

In [ ]:
res_pd

In [5]:
def get_num_with_contion(res_pd, key, values):
    if isinstance(values, list):
        total_pd = res_pd[res_pd[key].isin(values)]
    else:
        total_pd = res_pd[res_pd[key] == values]
        
    if len(res_pd) > 0:
        percent = "{:.2f}".format(len(total_pd) / len(res_pd) * 100)
    else:
        percent = None
    return percent

In [6]:
for module in inventory:
    use_num = get_num_with_contion(res_pd, module, True)
    nouse_num = get_num_with_contion(res_pd, module, False)
    print(module, use_num, nouse_num)

program_generator 70.84 29.16
program_verifier 70.84 29.16
program_executor 70.84 29.16
answer_generator 100.00 0.00
row_lookup 46.51 53.49
column_lookup 3.89 96.11
table_verbalizer 2.94 97.06
knowledge_retrieval 16.40 83.60
solution_generator 34.84 65.16
